<a href="https://colab.research.google.com/github/salvapineda/notebooks/blob/main/ExpansionPlanning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Economic Dispatch 

This Jupyter Notebook solves an instance of the Economic Dispatch Problem described in XXX

## Requirements

In [1]:
!pip install -q pyomo
!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64
import pyomo.environ as pe
import pandas as pd
import numpy as np
ipopt = pe.SolverFactory('ipopt', executable='/content/ipopt')

     |████████████████████████████████| 9.1 MB 4.2 MB/s 
     |████████████████████████████████| 49 kB 5.7 MB/s 


## Input Data

In [17]:
gen = pd.DataFrame({
       'unit': [1,   2], 
       'q_cost': [0.01,  0.02], 
       'l_cost': [10,  20],
       'pmax': [200, 200]})
inv_cost = 14
dem = [50,100,150]
ngen = len(gen)
ntime = len(dem)

## Solving the Expansion Planning Model

In [18]:
# Model
m = pe.ConcreteModel()
# Sets
m.g = pe.Set(initialize=list(range(ngen)))
m.t = pe.Set(initialize=list(range(ntime)))
# Variables
m.p = pe.Var(m.g,m.t,within=pe.NonNegativeReals)
m.cap = pe.Var(m.g,within=pe.NonNegativeReals)
m.alpha = pe.Var(m.g,m.t,within=pe.NonNegativeReals)
m.beta = pe.Var(m.g,m.t,within=pe.NonNegativeReals)
m.lamda = pe.Var(m.t)
def obj_rule(m):
  return sum((gen.loc[0,'q_cost']*gen.loc[0,'q_cost']-2*gen.loc[0,'q_cost'])*m.p[0,t]*m.p[0,t] + (inv_cost-m.beta[0,t])*m.cap[0] for t in m.t)
m.obj = pe.Objective(rule=obj_rule)
# Energy balance
def bal_rule(m,t):
  return sum(m.p[g,t] for g in m.g) == dem[t]
m.bal = pe.Constraint(m.t,rule=bal_rule)
# Maximum generation
def max_gen_rule(m,g,t):
  return m.p[g,t] <= m.cap[g]
m.max_gen = pe.Constraint(m.g,m.t, rule=max_gen_rule)
# Stationarity
def stat_rule(m,g,t):
  return 2*gen.loc[g,'q_cost']*m.p[g,t] + gen.loc[g,'l_cost'] - m.lamda[t] - m.alpha[g,t] + m.beta[g,t] == 0
m.stat = pe.Constraint(m.g,m.t,rule=stat_rule)
# Complementarity 1
def com1_rule(m,g,t):
  return m.p[g,t]*m.alpha[g,t] <= 0
m.com1 = pe.Constraint(m.g,m.t,rule=com1_rule)
# Complementarity 2
def com2_rule(m,g,t):
  return (m.cap[g]-m.p[g,t])*m.beta[g,t] <= 0
m.com2 = pe.Constraint(m.g,m.t,rule=com2_rule)
# Fix capacity unit 2
def fix_cap_rule(m):
  return m.cap[1] == gen.loc[1,'pmax']
m.fix_cap = pe.Constraint(rule=fix_cap_rule)
# We solve the optimization problem using IPOPT
pe.SolverFactory('ipopt').solve(m,tee=True).write()
print(m.cap[0].value)

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       31
Number of nonzeros in inequality constraint Jacobian.:       42
Number of nonzeros in Lagrangian Hessian.............:       21

Total number of variables............................:       23
                     variables with only lower bounds:       20
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Tot

## Solution Output

In [4]:
# Print results
print('obj =',m.obj())
print('p1 =',m.p[0].value)
print('p2 =',m.p[1].value)

obj = 0.0


KeyError: ignored